In [1]:
!pip install sacremoses
!pip3 install git+https://github.com/huggingface/transformers git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.3 MB/s eta 0:00:0000:010:01
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0x_o2c4x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0x_o2c4x
  Resolved https://github.com/huggingface/transformers to commit 297b732bdf25ca60df7ea624f06826f5b14aa6c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_zym6_02
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_zym6_02
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import os
import numpy as np
import json
from transformers import set_seed, GPT2Config, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification,pipeline
import pickle
import pandas as pd
import string

tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")
pipe = pipeline("token-classification", model=model,tokenizer = tokenizer, aggregation_strategy='simple')

tokenizer1 = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model1 = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")
pipe1 = pipeline("ner", model=model1, tokenizer=tokenizer1, aggregation_strategy="simple")


# class Magic_train
def remove_stopwords(data):
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
    'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
    'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 
    'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
     'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
     'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'help','do', 'does', 'did', 'doing',
     'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
    'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
    'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
    's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
     'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
    'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
     "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't",'dad', 'mom','father','mother','doctor', '’'] + list(string.punctuation)
    
    return set([x for x in data if x not in stopwords])
        
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def train_queries_ner(index,query):
    
    manually_added = {85:['nail'],224:['itchy'],298:['mole'],331:['cancerous'],339:['nose'],111:['infected'],178:['mole'],175:['pop','sensitive','warm'],
                     141:['head','mole'],42:['face','outside'],33:['hand','not itch']}
    if(index in manually_added.keys()):
        return manually_added[index]
    
    tokens0 = pipe(query)
    tokens1 = pipe1(query)
#     print(index,tokens0,tokens1)
    
    labels = []
    labels_unfiltered = []
    
    ##tokens0
    for tag in tokens0:
        labels_unfiltered.append(tag['word'].lower())
        if(tag['entity_group'] in ['DISEASE_DISORDER','BIOLOGICAL_STRUCTURE','SIGN_SYMPTOM','DETAILED_DESCRIPTION','MEDICATION']):
            labels.append(tag['word'].lower())
            
    ##tokens1
    tags = []
    j = -1
    groups = []
    for tag1 in tokens1:
        tag=tag1['word']
        if("##" in tag and j>=0):
            tags[j]+=tag[2:]
        else:
            tags.append(tag)
            groups.append(tag1['entity_group'])
            j+=1
    for j,tag in enumerate(tags):
        labels_unfiltered.append(tag.lower())
        if(groups[j] in ['Disease_disorder','Sign_symptom','Biological_structure','Coreference','Detailed_description','Color','Medication','Therapeutic_procedure','Shape']):
            labels.append(tag.lower())
#     print(groups)
    
    
    labels = remove_stopwords(labels)#set([x for x in labels if x not in stopwords])
#     if(labels == []):
#         labels = remove_stopwords(labels_unfiltered)
    
    return labels    
    
    
def preprocess_train(train_json_path,augmented_titles,augmented_contents,labels_path,labels_col_names,image_dir):
    read_labels = pd.read_csv(labels_path,names=labels_col_names)
    train_labels = {}
    for i,disease0 in enumerate(read_labels['disease0']):
        labels = []
        for col in labels_col_names:
            if(pd.isna(read_labels[col][i+1])):
                continue
            else:
                labels.append(read_labels[col][i+1].lower())
        train_labels[i]=labels
    
    
    list_images = os.listdir(image_dir)
    images_mapping = {}
    train_images = {}
    for image in list_images:
        train_images[image.split('.')[0]]=os.path.join(image_dir,image)
    
    
    trainf = open(train_json_path)
    train_data_json = json.load(trainf)
    titles = json.load(open(augmented_titles))
#     print(type(list(titles.keys())[0]))
    contents = json.load(open(augmented_contents))
    train_queries = {}
    train_answers = {}
    deleted_queries = []
    for i,data in enumerate(train_data_json):
        images_mapping[i]=data['encounter_id']
        queries = []
        query = ""
        if(data['query_title_en'] != "[deleted by user]"):
            queries.append(data['query_title_en'])
            for title in titles[str(i)]:
                queries.append(title)
            query += data['query_title_en']
        if(data['query_content_en'] != "[removed]" and data['query_content_en']!="[deleted]"):
            query += data['query_content_en']
            queries[0]+=data['query_content_en']
#             print(i,len(queries),len(contents[str(i)]))
            if(str(i) in contents.keys()):
                for j in range(min(len(contents[str(i)]),len(titles[str(i)]))):
                    queries[1+j]+=contents[str(i)][j]
        if(query != ""):
            train_queries[i] = queries
            train_answers[i] = data['responses'][0]['content_en']
        else:
            if(images_mapping[i] in train_images.keys()):
                print("No images no query ",i)
            deleted_queries.append(i)
    print("No query data in train ",deleted_queries)
    
    
    train_data = {}
    train_data_without_image = {}
    train_data_without_query_tags_with_image = {}
    train_data_without_query_tags_without_image = {}
    for i in train_queries.keys():
        if(len(train_labels[i])==0):
            continue
        queries_tokens = []
        queries = []
        for query in train_queries[i]:
            query_tokens = train_queries_ner(i,query)
            if(len(query_tokens) == 0):
#                 query_tokens = remove_stopwords(remove_emojis(query).split(" "))
                print("No query tokens in train {}, {}".format(i,query))
            else:
                queries.append(query)
                queries_tokens.append(query_tokens)
        if(images_mapping[i] in train_images.keys()):
            is_image = True
            train_data[i] = [queries,queries_tokens,is_image,train_images[images_mapping[i]],train_answers[i],train_labels[i],images_mapping[i]]
        else:
            is_image = False
            train_data[i] = [queries,queries_tokens,is_image,None,train_answers[i],train_labels[i],images_mapping[i]]
    
    
    return train_data

def valid_queries_ner(index,query):
    
#     manually_added = {85:['nail'],224:['itchy'],298:['mole'],331:['cancerous'],339:['nose'],111:['infected'],178:['mole'],175:['pop','sensitive','warm'],
#                      141:['head','mole'],42:['face','outside'],33:['hand','not itch']}
#     if(index in manually_added.keys()):
#         return manually_added[index]
    
    tokens0 = pipe(query)
    tokens1 = pipe1(query)
#     print(index,tokens0,tokens1)
    
    labels = []
    labels_unfiltered = []
    
    ##tokens0
    for tag in tokens0:
        labels_unfiltered.append(tag['word'].lower())
        if(tag['entity_group'] in ['DISEASE_DISORDER','BIOLOGICAL_STRUCTURE','SIGN_SYMPTOM','DETAILED_DESCRIPTION','MEDICATION']):
            labels.append(tag['word'].lower())
            
    ##tokens1
    tags = []
    j = -1
    groups = []
    for tag1 in tokens1:
        tag=tag1['word']
        if("##" in tag and j>=0):
            tags[j]+=tag[2:]
        else:
            tags.append(tag)
            groups.append(tag1['entity_group'])
            j+=1
    for j,tag in enumerate(tags):
        labels_unfiltered.append(tag.lower())
        if(groups[j] in ['Disease_disorder','Sign_symptom','Biological_structure','Coreference','Detailed_description','Color','Medication','Therapeutic_procedure','Shape']):
            labels.append(tag.lower())
#     print(groups)
    
    
    labels = remove_stopwords(labels)#set([x for x in labels if x not in stopwords])
#     if(labels == []):
#         labels = remove_stopwords(labels_unfiltered)
    
    return labels


def preprocess_valid(valid_json_path,labels_path,labels_col_names,image_dir):
    read_labels = pd.read_csv(labels_path,names=labels_col_names)
    valid_labels = {}
    for i,disease0 in enumerate(read_labels['disease0']):
        labels = []
        for col in labels_col_names:
            if(pd.isna(read_labels[col][i+1])):
                continue
            else:
                labels.append(read_labels[col][i+1].lower())
        valid_labels[i]=labels
    
    
    list_images = os.listdir(image_dir)
    images_mapping = {}
    valid_images = {}
    for image in list_images:
        valid_images[image.split('.')[0]]=os.path.join(image_dir,image)
    
    
    validf = open(valid_json_path)
    valid_data_json = json.load(validf) 
    valid_queries = {}
    valid_answers = {}
    deleted_queries = []
    for i,data in enumerate(valid_data_json):
        images_mapping[i]=data['encounter_id']
        query = ""
        if(data['query_title_en'] != "[deleted by user]"):
            query += data['query_title_en']
        if(data['query_content_en'] != "[removed]" and data['query_content_en']!="[deleted]"):
            query += data['query_content_en']
        if(query != ""):
            valid_queries[i] = remove_emojis(query)
            valid_answers[i] = data['responses'][0]['content_en']
        else:
            if(images_mapping[i] in valid_images.keys()):
                print("No query no image in valid ",i)
            deleted_queries.append(i)
    print("No query data in valid",deleted_queries)
            
    
    valid_data = {}
    valid_data_without_image = {}
    valid_data_without_query_tags_with_image = {}
    valid_data_without_query_tags_without_image = {}
    for i in valid_queries.keys():
        query_tokens = valid_queries_ner(i,valid_queries[i])
        if(len(query_tokens)==0):
            query_tokens = remove_stopwords(remove_emojis(valid_queries[i]).split(" "))
            print("No query tokens in valid {}".format(i))
        if(images_mapping[i] in valid_images.keys()):
            is_image = True
            valid_data[i] = [valid_queries[i],query_tokens,is_image,valid_images[images_mapping[i]],valid_answers[i],valid_labels[i],images_mapping[i]]
        else:
            is_image = False
            valid_data[i] = [valid_queries[i],query_tokens,is_image,None,valid_answers[i],valid_labels[i],images_mapping[i]]
    
    
    return valid_data

def test_queries_ner(index,query):
    
#     manually_added = {85:['nail'],224:['itchy'],298:['mole'],331:['cancerous'],339:['nose'],111:['infected'],178:['mole'],175:['pop','sensitive','warm'],
#                      141:['head','mole'],42:['face','outside'],33:['hand','not itch']}
#     if(index in manually_added.keys()):
#         return manually_added[index]
    
    tokens0 = pipe(query)
    tokens1 = pipe1(query)
    
    labels = []
    labels_unfiltered = []
    
    ##tokens0
    for tag in tokens0:
        labels_unfiltered.append(tag['word'].lower())
        if(tag['entity_group'] in ['DISEASE_DISORDER','BIOLOGICAL_STRUCTURE','SIGN_SYMPTOM','DETAILED_DESCRIPTION','MEDICATION']):
            labels.append(tag['word'].lower())
            
    ##tokens1
    tags = []
    j = -1
    groups = []
    for tag1 in tokens1:
        tag=tag1['word']
        if("##" in tag and j>=0):
            tags[j]+=tag[2:]
        else:
            tags.append(tag)
            groups.append(tag1['entity_group'])
            j+=1
    for j,tag in enumerate(tags):
        labels_unfiltered.append(tag.lower())
        if(groups[j] in ['Disease_disorder','Sign_symptom','Biological_structure','Coreference','Detailed_description','Color','Medication','Therapeutic_procedure','Shape']):
            labels.append(tag.lower())
#     print(groups)
    
    
    labels = remove_stopwords(labels)
    
    return labels

def preprocess_test(test_json_path,image_dir):
    list_images = os.listdir(image_dir)
    images_mapping = {}
    test_images = {}
    for image in list_images:
        test_images[image.split('.')[0]]=os.path.join(image_dir,image)
    
    
    testf = open(test_json_path)
    test_data_json = json.load(testf) 
    test_queries = {}
    deleted_queries = []
    deleted_query_image = []
    for i,data in enumerate(test_data_json):
        images_mapping[i]=data['encounter_id']
        query = ""
        if(data['query_title_en'] != "[deleted by user]"):
            query += data['query_title_en']
        if(data['query_content_en'] != "[removed]" and data['query_content_en']!="[deleted]"):
            query += data['query_content_en']
        if(query != ""):
            test_queries[i] = remove_emojis(query)
        else:
            if(images_mapping[i] not in test_images.keys()):
                print("{} does not have image nor query".format(i))
                deleted_query_image.append(i)
            deleted_queries.append(i)
    print("queries without any title ",deleted_queries)
            
    
    test_data = {}
    test_data_without_image = {}
    test_data_without_query_tags_with_image = {}
    test_data_without_query_tags_without_image = {}
    all_tokens = {}
    for i in test_queries.keys():
        query_tokens = test_queries_ner(i,test_queries[i])
        if(len(query_tokens) == 0):
            query_tokens = remove_stopwords(remove_emojis(test_queries[i]).split(" "))
            print("No query tokens in test {}".format(i))
        all_tokens[i]=[query_tokens,test_queries[i]]
        if(images_mapping[i] in test_images.keys()):
            is_image = True
            test_data[i] = [test_queries[i],query_tokens,is_image,test_images[images_mapping[i]],images_mapping[i]]
        else:
            is_image = False
            test_data_without_image[i] = [test_queries[i],query_tokens,is_image,None,images_mapping[i]]
    for i in all_tokens.keys():
        print(i,all_tokens[i])
    
    return test_data

2024-05-09 12:28:17.099826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 12:28:17.099956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 12:28:17.222182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

In [3]:
import pickle
train_data = preprocess_train('/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/train_downloaded.json',
                              '/kaggle/input/mediqa-magic/augmented_titles.json','/kaggle/input/mediqa-magic/augmented_contents.json',
                                '/kaggle/input/mediqa-magic/Train_answers_labels.csv',['disease0', 'disease1', 'disease2', 'disease3', 'disease4'],
                                '/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/images/train')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


No query data in train  [9, 23, 24, 36, 54, 61, 62, 65, 75, 88, 95, 108, 112, 126, 128, 143, 152, 153, 155, 161, 164, 167, 184, 199, 208, 218, 230, 237, 246, 252, 261, 284, 288, 291, 299, 300, 316, 341]
No query tokens in train 11, What are our thoughts as we approach one month of being hit with them?What are some reasons we should not panic?
No query tokens in train 18, Can anyone suggest what this might be? (Details in comments)
No query tokens in train 18, Could anyone suggest what this might be? (Details in comments)Is there anywhere I can get access to my data?
No query tokens in train 18, Does anyone suggest what this might be?What is your personal opinion on climate change?
No query tokens in train 21, What are some facts about Staph?What are the advantages or disadvantages of a lawyer?
No query tokens in train 21, What is the Staph?Are there any opportunities for our organization to grow and prosper?
No query tokens in train 27, Should I get this checked out? I’ve had it for se

In [4]:
valid_for_training = preprocess_train('/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/valid_downloaded.json',
                                        '/kaggle/input/mediqa-magic/augmented_valid_titles.json','/kaggle/input/mediqa-magic/augmented_valid_contents.json',
                                      '/kaggle/input/mediqa-magic/valid_answers_lables.csv',['disease0', 'disease1', 'disease2'],
                                '/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/images/valid')
valid_data = preprocess_valid('/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/valid_downloaded.json',
                                '/kaggle/input/mediqa-magic/valid_answers_lables.csv',['disease0', 'disease1', 'disease2'],
                                '/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/images/valid')
test_data = preprocess_test('/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/test_downloaded.json',
                                '/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release/images/test')

No query data in train  [13, 31]
No query tokens in train 4, Have you ever had any cancer or an autoimmune disease?If I believe in the power of music, I can not change my views on a message. Which is better, why?
No query tokens in train 10, Can you help me identify this?What is the ideal length and duration of a study?
No query tokens in train 20, Does it have an ant on it, and should I be worried?What is your view on the recent Supreme Court decision?
No query tokens in train 20, This is from an ant. Should I be worried?What is your view on the issue of Ezra Lawn | From the bottom of my heart.
No query tokens in train 27, What skin problem is this?
No query tokens in train 27, What is your skin problem?What are some good or bad ways to live a successful American life?
No query tokens in train 27, What is the skin problem?What are the opportunities to become friends with each other?
No query tokens in train 27, What are skin problems?What is the best way to explain the whole situation

In [5]:
for key in valid_for_training.keys():
    train_data[347+key] = valid_for_training[key]
    

# for tags in data_val['answer_tokens']:
#     for tag in tags:
#         answer_vocab.add(tag)

In [6]:
answer_vocab = set()
for i in train_data.keys():
    for tag in train_data[i][5]:
        answer_vocab.add(tag)
        
print(len(answer_vocab))

155


In [7]:
import itertools
import random

def prepare_training_data(train_data,lower_bound,upper_bound,max_sample_per_example,answer_vocab):
    data = {'queries':[],'query_tokens':[],'is_image':[],'img_paths':[],'answers':[],'answer_tokens':[],"neg_tokens":[],'img_ids':[]}
    for i in train_data.keys():
        df = train_data[i]
        pos_answer_tokens = df[5]
        if(len(pos_answer_tokens)==0):
            print("Here ",i)
            print(pos_answer_tokens)
        neg_answer_tokens = []
        for token in answer_vocab:
            if(token not in pos_answer_tokens):
                neg_answer_tokens.append(token)
        for j in range(len(df[0])):
            data['queries'].append(df[0][j])
            data['query_tokens'].append(df[1][j])
            data["is_image"].append(df[2])
            data['img_paths'].append(df[3])
            data['answers'].append(df[4])
            data['answer_tokens'].append(pos_answer_tokens[j%len(pos_answer_tokens)])
            data["neg_tokens"].append(random.choice(neg_answer_tokens))
            data['img_ids'].append(df[6])
#         continue

#         random_numbers = random.choices(range(lower_bound+1, min(len(df[1]),upper_bound)+1), k=max_sample_per_example)
#         combinations = []
#         for r in random_numbers:
#             l = random.sample(df[1],r)
#             combinations.append(l)
        
#         for j,combination in enumerate(combinations):
#             data['queries'].append(df[0])
#             data['query_tokens'].append(combination)
#             data["is_image"].append(df[2])
#             data['img_paths'].append(df[3])
#             data['answers'].append(df[4])
#             data['answer_tokens'].append(pos_answer_tokens[j%len(pos_answer_tokens)])
#             data["neg_tokens"].append(random.choice(neg_answer_tokens))
#             data['img_ids'].append(df[6])
            
    return data

def prepare_valid_data(valid_data,upper_bound):
    data = {'queries':[],'query_tokens':[],'is_image':[],'img_paths':[],'answers':[],'answer_tokens':[],'img_ids':[]}
    for i in valid_data.keys():
        df = valid_data[i]
        if(len(df[1])<=upper_bound):
            data['queries'].append(df[0])
            data['query_tokens'].append(df[1])
            data["is_image"].append(df[2])
            data['img_paths'].append(df[3])
            data['answers'].append(df[4])
            data['answer_tokens'].append(df[5])
            data['img_ids'].append(df[6])
        else:
            tokens = random.sample(df[1],upper_bound)
            data['queries'].append(df[0])
            data['query_tokens'].append(tokens)
            data["is_image"].append(df[2])
            data['img_paths'].append(df[3])
            data['answers'].append(df[4])
            data['answer_tokens'].append(df[5])
            data['img_ids'].append(df[6])
            
    return data

def prepare_test_data(test_data,upper_bound):
    data = {'queries':[],'query_tokens':[],'is_image':[],'img_paths':[],'img_ids':[]}
    for i in test_data.keys():
        df = test_data[i]
        if(len(df[1])<=upper_bound):
            data['queries'].append(df[0])
            data['query_tokens'].append(df[1])
            data["is_image"].append(df[2])
            data['img_paths'].append(df[3])
            data['img_ids'].append(df[4])
        else:
            tokens = random.sample(df[1],upper_bound)
            data['queries'].append(df[0])
            data['query_tokens'].append(tokens)
            data["is_image"].append(df[2])
            data['img_paths'].append(df[3])
            data['img_ids'].append(df[4])
            
    return data
    

In [8]:
data_train = prepare_training_data(train_data,3,20,8,answer_vocab)
data_val = prepare_valid_data(valid_data,20)
data_test = prepare_test_data(test_data,20)
        
with open('train_data.pkl', 'wb') as f:
    pickle.dump(data_train, f)     

with open('val_data.pkl', 'wb') as f:
    pickle.dump(data_val, f)     
    
with open('test_data.pkl', 'wb') as f:
    pickle.dump(data_test, f)     

/tmp/ipykernel_34/296144314.py:80: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  tokens = random.sample(df[1],upper_bound)


In [9]:
from transformers import BertTokenizer, BertModel
import torch
import skimage.io as io
import skimage.transform as transform
import torchvision
import clip
import pandas as pd
from PIL import Image
import pickle
import json
import os
from tqdm import tqdm
import argparse
import string
import random
import numpy as np
import csv
from transformers import set_seed, GPT2Config, GPT2Tokenizer
from tqdm import tqdm
import sys
import torch
import torch.nn as nn
from transformers import set_seed, GPT2Config, GPT2Tokenizer
from transformers import AutoTokenizer
from transformers.models.biogpt import BioGptTokenizer
import os
import pandas as pd
from torch.utils.data import Dataset
import pickle
from torch.utils.data import DataLoader, random_split
import numpy as np
import pdb

class magicDataset(Dataset):
    def __init__(self, path,answer_vocab, split='train',like_test=False):
        device = "cuda:0"
        super().__init__()
        data_path = path
        with open(data_path, 'rb') as f:
            data = pickle.load(f)
        sys.stdout.flush()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.text_model = BertModel.from_pretrained('bert-base-uncased')
        self.device = torch.device('cuda:0')
#         self.tokenizer.pad_token = self.tokenizer.eos_token

        self.img_ids = data["img_ids"]
        self.is_image = data["is_image"]
        self.queries = data["queries"]
        self.query_tokens = data["query_tokens"]
#         self.questions = data['questions']
        self.img_paths = data['img_paths']
        if(split!='test'):
            self.answers = data['answers']
            self.answer_tokens = data['answer_tokens']
            if(split!='valid'):
                self.neg_tokens = data["neg_tokens"]
        self.clip_model, self.preprocess = clip.load("ViT-B/32", device=device, jit=False)

        self.max_query_len = 40
        self.labels = answer_vocab       
        self.train_setting = True if (split=='train') else False
        self.valid_setting = True if (split=="valid") else False
#         self.prefix_len = prefix_length
        
    def __len__(self):
        return len(self.queries)
    
    
    def prepare_image(self,img_path):
        with torch.no_grad():
            prefix_i = self.clip_model.encode_image(self.preprocess(Image.open(img_path)).unsqueeze(0).to(self.device)).cpu()
        return prefix_i
        
    def pad_query_tokens(self,query_tokens):
        input_sequence = ""
        for token in query_tokens:
            input_sequence += token
            input_sequence += " "
            
        input_sequence = input_sequence[:-1]
#         tokenized_inputs = self.tokenizer(input_sequence, padding='max_length', truncation=True, max_length=self.max_query_len, return_tensors='pt')
#         input_ids = tokenized_inputs['input_ids']
#         attention_masks = tokenized_inputs['attention_mask']
        
#         with torch.no_grad():
#             outputs = self.text_model(input_ids, attention_mask=attention_masks)
#         output_embeddings = outputs.last_hidden_state
        with torch.no_grad():
            output_embeddings = self.clip_model.encode_text(clip.tokenize(query_tokens).to(self.device))
        
        return output_embeddings #input_ids,attention_masks,
    
    def prepare_answer_labels(self,answer_tokens):
        labels = list(self.labels)
#         print(labels,answer_tokens)
        label_index = labels.index(answer_tokens)
        return label_index
    def prepare_answer_labels_valid(self,answer_tokens):
        binary_labels = np.array([1 if label in answer_tokens else 0 for label in self.labels])
        return binary_labels
            
    def __getitem__(self, index):
        img_id = self.img_ids[index]
#         is_image = self.is_image[index]
        if(self.img_paths[index]!=None):
            img_prefix = self.prepare_image(self.img_paths[index]).squeeze(0)
#             print(img_prefix.shape)
            is_image = True
        else:
            img_prefix = torch.zeros((512))
            is_image = False
#         img_prefix = self.prepare_image(self.img_paths[index]).squeeze(0)
        query_embeddings = self.pad_query_tokens(self.query_tokens[index])
#         query_embeddings = query_embeddings.squeeze(0)
#         query_token = list(self.query_tokens[index])
#         while(len(query_token)< 50):
#             query_token.append("__")
        if(self.train_setting):
            answer_labels = self.prepare_answer_labels(self.answer_tokens[index])
            neg_labels = self.prepare_answer_labels(self.neg_tokens[index])
            return img_id,[is_image],img_prefix,query_embeddings, answer_labels,neg_labels
        elif(self.valid_setting):
            answer_labels = self.prepare_answer_labels_valid(self.answer_tokens[index])
            return img_id,[is_image],img_prefix,query_embeddings, answer_labels
        
        return img_id,[is_image],img_prefix, query_embeddings

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 for bidirectional
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # *2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
#         print(out.shape)
        out1 = self.fc(out[:, -1, :])
#         out1 = F.softmax(out1,dim=1)
        return out1

class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
#         self.fc0 = nn.Linear(input_size,512)
        self.fc1 = nn.Linear(input_size, 256)  # Input layer
        self.fc2 = nn.Linear(256, 256)          # Hidden layer
        self.fc3 = nn.Linear(256, num_classes)  # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU activation to the first fully connected layer
        x = F.relu(self.fc2(x))  # Apply ReLU activation to the second fully connected layer
        x = self.fc3(x)          # Output layer (no activation function for multi-class classification)
#         x = F.softmax(x,dim=1)
        return x


In [11]:
class Combinedmodel(nn.Module):
    def forward(self, vision_embeds,text_embeds,is_image):
#         bilst_hidden,bilstm_output = self.bilstm_layer(text_embeds)
        text_embeds = text_embeds.to(torch.float32)
#         print("is_image ",is_image[0])
        text_outputs = self.bilstm_layer(text_embeds)
        
        if(is_image[0] == True):
#             print("Here when true")
            vision_embeds = vision_embeds.to(torch.float32)
            vision_outputs = self.vision_mlp(vision_embeds)
        else:
            vision_outputs = vision_embeds
        return text_outputs,vision_outputs
        
    def __init__(self, prefix_len,embedding_dim, num_labels):
        super(Combinedmodel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.text_model = BertModel.from_pretrained('bert-base-uncased')
        self.bilstm_layer = BiLSTM(embedding_dim, 256, 2, embedding_dim)
        self.vision_mlp = MLP(embedding_dim,embedding_dim)
#         self.vision_mlp = MLP(prefix_len,num_labels)
        
    

In [12]:
import torch
import clip
from PIL import Image

In [13]:
train_dataset = magicDataset('train_data.pkl',answer_vocab, split='train',like_test=False)
val_dataset = magicDataset('val_data.pkl',answer_vocab, split='valid',like_test=False)
test_dataset = magicDataset('test_data.pkl',answer_vocab, split='test',like_test=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|███████████████████████████████████████| 338M/338M [00:11<00:00, 31.6MiB/s]


In [14]:
labels_embeddings = {}
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
labels_list = list(answer_vocab)
# for label in answer_vocab:
with torch.no_grad():
    text = clip.tokenize(list(answer_vocab)).to(device)
    labels_embeddings = clip_model.encode_text(text)
    
print(labels_embeddings.shape)

torch.Size([155, 512])


In [15]:
import pandas as pd
df = pd.read_csv('/kaggle/input/mediqa-magic/labels-mapping.csv')
disease_mapping = {}
print(len(df['Common description']),len(df['Disease']))
# print(df['Disease'][0])
for i,disease in enumerate(df['Disease']):
    if(disease.lower() in disease_mapping.keys()):
        print(disease.lower())
        continue
    disease_mapping[disease.lower()] = df['Common description'][i]
# print(disease_mapping['cyst'])
print(len(disease_mapping.keys()))

160 160
160


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
# def triplet_loss()
triplet_loss = (nn.TripletMarginWithDistanceLoss(distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y,dim=1)))

In [18]:
import torch
train_labels = data_train["answer_tokens"]
print(len(train_labels))
labels = []
for answer_tokens in train_labels:
    labels.append([1 if label in answer_tokens else 0 for label in answer_vocab])
print(len(labels))
class_counts = torch.tensor(labels).sum(dim=0)
print(class_counts)
total_samples = len(data_train["answer_tokens"])
class_weights = total_samples / (2 * class_counts)
print(sum(class_weights),class_weights.sum())
class_weights/= sum(class_weights)
class_weights = class_weights.to(device)

1824
1824
tensor([ 54,   2,   3,   2,   5,   6,   3,  20,   3,  53,  12,  85,   6,  11,
         36,   6,   2,  12,   8,  12,  18,  48,  17,   8,  40,   6,   6,   8,
          3,   2,   3,   1,   3,  18,   7,   2,   3,  13,   2,   2,   4,  37,
         20,  33,   2,   2, 102,   2,  16,  53,  12,   5,   3,   3,  14,   2,
          3,   6,  21,   1,   4,   2,  30,   2,  39,  11,  18,   6,   2,  21,
         24,   3,   7,   4,   7,  12,   2,   5,   5, 227,   6,   6,  33,   3,
         12,   6,  66,   3,   5,   3,   6,   7,   6,   6,  11,   2,   6,   3,
          3,   6,   5,  12,   2,  16,  17,  14,   4,  22,   2,   6,  61,   4,
          5,   6,  18,  32,   3,  18,   1,   6,   3,  10,   9,   6,   8,   5,
          6,   6,   3,  11,   3,   3,   9,   1,   8,   5,  23,   6,  16,   3,
          3,  14,  21,   2,   3,  40,   6,  15,  19,  12,   8,  68,  16,   6,
          1])
tensor(31061.1641) tensor(31061.1621)


In [26]:
import torch.nn.functional as F
from sklearn.metrics import f1_score
import pickle

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU")

batch_size = 1
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=False, drop_last=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False, drop_last=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
epochs = 5
learning_rate = 1e-4
warmup_steps = 500

model = Combinedmodel(512,512,len(answer_vocab))
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=epochs * len(train_dataloader),
    )

# f1_save = 0
# model = Combinedmodel(512,768,len(answer_vocab))
# model = model.to(device)
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# scheduler = get_linear_schedule_with_warmup(
#         optimizer,
#         num_warmup_steps=warmup_steps,
#         num_training_steps=epochs * len(train_dataloader),
#     )

# f1_save = 0
# # class_weights = class_weights.to(device)
# loss_fn = nn.BCEWithLogitsLoss()#weight=class_weights)

CUDA is available


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
print(labels_embeddings.shape)
print(class_weights.shape)

torch.Size([155, 512])
torch.Size([155])


In [ ]:
f1_save =0 
for epoch in range(epochs):
    with tqdm(total=batch_size * len(train_dataloader)) as epoch_pbar:
        model.train()
        epoch_pbar.set_description(f"Epoch {epoch}")
        total_loss = 0
#                img_id,[is_image],img_prefix,query_embeddings, answer_labels,neg_labels
        for i,(img_id,is_image,img_prefix,query_embeddings, answer_labels,neg_label) in enumerate(train_dataloader):
            img_prefix = img_prefix.to(device)
            query_embeddings = query_embeddings.to(device)
#             print("i ",i)
#             print("query_embeddings ",query_embeddings.shape)
            text_outputs,image_outputs = model(img_prefix,query_embeddings,is_image)
#             print("text_outputs ",text_outputs.shape)
#             print("labels_embeddings[answer_labels[0]] ",labels_embeddings[answer_labels[0]].shape)
#             print(F.cosine_similarity(text_outputs, labels_embeddings[answer_labels[0]].unsqueeze(0),dim=1))
            
            loss = class_weights[answer_labels[0]]*triplet_loss(text_outputs, labels_embeddings[answer_labels[0]].unsqueeze(0), labels_embeddings[neg_label[0]].unsqueeze(0))
#             print(output)
#             
            
#             labels = torch.tensor(answer_labels).to(device)
#             print(text_outputs.dtype)
#             print(labels.dtype)
#             loss = loss_fn(text_outputs[0,:],labels[0,:])
            
            
#             print(text_outputs.shape)
#             print(text_outputs.expand_as(labels_embeddings).shape)
#             print(labels_embeddings.shape)
#             print(labels.squeeze(0).shape)
#             loss = multi_class_cosine_similarity_loss(text_outputs, labels_embeddings, labels)
            if(is_image[0]==True):
                loss+= class_weights[answer_labels[0]]*triplet_loss(image_outputs, labels_embeddings[answer_labels[0]].unsqueeze(0), labels_embeddings[neg_label[0]].unsqueeze(0))
            loss.backward(retain_graph=True)

# #                 loss = ContrastiveLoss(image_outputs[0,:],labels[0,:])
#                 loss+=torch.mean(loss_fn(image_outputs.expand_as(labels_embeddings), labels_embeddings, labels.squeeze(0)))

#             loss/=text_outputs.shape[0]
#             print(loss)
#             optimizer.zero_grad()
#             loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            total_loss+=loss.item()
            epoch_pbar.update(img_prefix.shape[0])
    predicted_probs = []
    true_labels = []
    for i,(img_id,is_image,img_prefix, query_embeddings, answer_labels) in enumerate(val_dataloader):
        img_prefix = img_prefix.to(device)
        query_embeddings = query_embeddings.to(device)
        text_outputs,image_outputs = model(img_prefix,query_embeddings,is_image)
        cos_sim_text = F.cosine_similarity(text_outputs.expand_as(labels_embeddings), labels_embeddings, dim=1)
        cos_sim = cos_sim_text

        if(is_image[0]==True):
            cos_sim_image = F.cosine_similarity(image_outputs.expand_as(labels_embeddings), labels_embeddings, dim=1)
            cos_sim+=cos_sim_image
            cos_sim/=2
        predicted_probs.append(cos_sim.detach().to("cpu"))
        for label in answer_labels:
            true_labels.append(label)

    print(max(predicted_probs[0]))
    predicted_probs = np.array(predicted_probs)
    true_labels = np.array(true_labels)
    threshold = 0.05
    max_f1 = 0
    while(threshold<0.2):
#         print(predicted_probs)
        pred_labels = (predicted_probs > threshold).astype(int)
        pt = torch.tensor(pred_labels)
        tt = torch.tensor(true_labels)
        f1 = f1_score(tt, pt, average='samples')
        print("on Epoch no. {}, with Threshold: {} ,F1 score: {}".format(epoch,threshold, f1))
        max_f1=max(max_f1,f1)
        threshold+=0.002
        
    if(max_f1>f1_save):
        f1_save = max_f1
        out_path ='model_with_highest_f1_score.pt'
        torch.save(model.state_dict(), out_path)
    

Epoch 0: 100%|██████████| 1824/1824 [03:49<00:00,  7.95it/s]


tensor(0.2141)
on Epoch no. 0, with Threshold: 0.05 ,F1 score: 0.027524164851198893
on Epoch no. 0, with Threshold: 0.052000000000000005 ,F1 score: 0.02837734484513336
on Epoch no. 0, with Threshold: 0.054000000000000006 ,F1 score: 0.029292078876076034
on Epoch no. 0, with Threshold: 0.05600000000000001 ,F1 score: 0.029812002495899895
on Epoch no. 0, with Threshold: 0.05800000000000001 ,F1 score: 0.030009370691171183
on Epoch no. 0, with Threshold: 0.06000000000000001 ,F1 score: 0.031149300374156613
on Epoch no. 0, with Threshold: 0.06200000000000001 ,F1 score: 0.031234206680988492
on Epoch no. 0, with Threshold: 0.06400000000000002 ,F1 score: 0.029468362566903782
on Epoch no. 0, with Threshold: 0.06600000000000002 ,F1 score: 0.029847929925584844
on Epoch no. 0, with Threshold: 0.06800000000000002 ,F1 score: 0.029516992379535163
on Epoch no. 0, with Threshold: 0.07000000000000002 ,F1 score: 0.030665520563158438
on Epoch no. 0, with Threshold: 0.07200000000000002 ,F1 score: 0.0304624808

Epoch 1:   4%|▎         | 67/1824 [00:07<03:31,  8.31it/s]

In [29]:
# class_weights[91]

tensor(0.0042, device='cuda:0')

In [21]:
# for epoch in range(epochs):
predictions = []
img_ids = []
predicted_probs = []
true_labels = []
with tqdm(total=batch_size * len(test_dataloader)) as epoch_pbar:
#     epoch_pbar.set_description(f"Epoch {epoch}")
    total_loss = 0
    for i,(img_id,is_image,img_prefix, query_embeddings) in enumerate(test_dataloader):
#         print(img_prefix.shape)
#         print(query_embeddings.shape)
        img_prefix = img_prefix.to(device)
        query_embeddings = query_embeddings.to(device)
        text_outputs,image_outputs = model(img_prefix,query_embeddings,is_image)
        
#         print(embeddings.shape)
#         print(labels_embeddings.shape)
        cos_sim_text = F.cosine_similarity(text_outputs.expand_as(labels_embeddings), labels_embeddings, dim=1)
        cos_sim = cos_sim_text

        if(is_image[0]==True):
            cos_sim_image = F.cosine_similarity(image_outputs.expand_as(labels_embeddings), labels_embeddings, dim=1)
            cos_sim+=cos_sim_image
            cos_sim/=2
        predicted_probs.append(cos_sim.detach().to("cpu"))
        print(labels_list[torch.argmax(cos_sim).item()])
        json_obj = {}
        json_obj["encounter_id"] = img_id[0]
        json_obj["responses"]=[{"content_en":disease_mapping[labels_list[torch.argmax(cos_sim).item()]]}]
        img_ids.append(img_id[0])
        predictions.append(json_obj)
        
#             outputs = model(img_prefix,query_embeddings,is_image)
#             labels = torch.tensor(answer_labels).to(device).float()
#             class_weights = class_weights.to(device)
#             loss = 0
#             for b in range(batch_size):
#                 loss+= loss_fn(outputs[b,:], labels[b,:])#, reduction='mean')
#             loss/=outputs.shape[0]
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()
#             total_loss+=loss
#             epoch_pbar.update(img_prefix.shape[0])

    
# dir_path = "/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release"
# # train_json = os.path.join(dir_path,"train_downloaded.json")
# valid_json = os.path.join(dir_path,"test_downloaded.json")
# # trainf = open(train_json)
# # train_data = json.load(trainf)
# validf = open(valid_json)
# valid_data = json.load(validf)
# # print(img_ids)
# references = []
# for data in valid_data:
# #     valid_ids.append(data["encounter_id"])
# #     print(data["encounter_id"])
#     if(data["encounter_id"] in img_ids):
#         json_obj = {}
#         json_obj["encounter_id"]=data["encounter_id"]
#         json_obj["responses"]=data["responses"]
#         references.append(json_obj)

  0%|          | 0/78 [00:00<?, ?it/s]

eczema
mole
eczema
spider veins
eczema
acne
acne
eczema
nail trauma
eczema
sun exposure
nail trauma
eczema
bruise
nail trauma
eczema
mole
eczema
eczema
scar
trauma
acne
inflammation of the nail fold
eczema
eczema
mole
lip dryness
eczema
mole
eczema
fungal infection
scar
scar
eczema
nail trauma
acne
eczema
eczema
mole
eczema
acne
eczema
acne
mole
eczema
mole
eczema
eczema
lip dryness
eczema
eczema
eczema
mole
mole
eczema
eczema
acne
eczema
eczema
bruise
tan
nail trauma
eczema
eczema
eczema
mole
lip licker's dermatitis
eczema
mole
acne
acne
eczema
athlete's foot
mole
eczema
eczema


  0%|          | 0/78 [00:10<?, ?it/s]

eczema
eczema due to dry skin


In [ ]:
test_ids = []
dir_path = "/kaggle/input/mediqa-magic/magic-imageclef2024/mediqa-magic-imageclef2024-release"
test_json = os.path.join(dir_path,"test_downloaded.json")
testf = open(test_json)
test_data = json.load(testf)
# print(img_ids)
all_prediction = []
for data in test_data:
    test_ids.append(data["encounter_id"])
    flag = False
    for pred in predictions:
        if(pred["encounter_id"] == data["encounter_id"]):
            all_prediction.append(pred)
            flag = True
            break
    if(flag==False):
        json_obj = {}
        json_obj["encounter_id"]=data["encounter_id"]
        json_obj["responses"] = [{"content_en":disease_mapping["dermatologist for lesion examination"]}]
        all_prediction.append(json_obj)

print(len(all_prediction))
print(len(test_data))

In [ ]:
# file_path = "reference.json"

# # Write the data to the JSON file
# with open(file_path, "w") as json_file:
#     json.dump(references, json_file, indent=4)
    
file_path = "prediction.json"

# Write the data to the JSON file
with open(file_path, "w") as json_file:
    json.dump(all_prediction, json_file, indent=4)